In [36]:
from PyPDF2 import PdfReader
from pdfminer.high_level import extract_text as fallback_text_extraction
import re
from bs4 import BeautifulSoup  
import requests
import urllib3
import pandas as pd
http = urllib3.PoolManager()
import matplotlib.pyplot as plt

years = ["2018", "2019", "2020", "2021"]
filenames = ['espn_' + year + ".pdf" for year in years] 
outcomes_files = [ "yearly/" + year + ".csv" for year in years]

def pdf_to_text(filename:str) -> str:

    text = ""
    try:
        reader = PdfReader(filename)
        for page in reader.pages:
            text += page.extract_text()
            return text
    except Exception as exc:
        text = fallback_text_extraction(filename)
        print("FAIL Parsing PDF ", exc)
        return


def parse_top_rb_from_espn_cheatsheet(text: str) -> list:
    #seperate text into a list of rows
    lines = []
    line = ""
    for char in text:
        if char == "\n":
            lines.append(line)
            line=""
            continue
        line+=char
    #Find runningback value in each row
    running_back_names = []
    for line in lines:
        line_split =  line.split('.')
        #index 2 in each line is the running back column
        #'6. (79) Aaron Rodgers, GB $4 13 6. (8) Ezekiel Elliott, DAL $50 7 83. (276) Dwayne Washington, NO $0 6 73. (164) Amon-Ra St. Brown, DET $0 9'
        running_back_match = re.search(r"\) .+(?=,)", line_split[2]).group()
        running_back_names.append(running_back_match[2:])
        if line == lines[20]:
            break
    return running_back_names


In [37]:
df = pd.DataFrame(columns=['name', 'cheatsheet_name', 'year', 'fantasy_points'])
for filename in filenames:
    text = pdf_to_text(filename)

In [38]:
rb_list = parse_top_rb_from_espn_cheatsheet(text)

In [42]:
lines=[]
line = ""
for char in text:
    if char == "\n":
        lines.append(line)
        line=""
        continue
    line+=char
#Find runningback value in each row
running_back_names = []
ADP = []
for line in lines:
    line_split =  line.split('.')
    print(line_split[2])
    #index 2 in each line is the running back column
    #'6. (79) Aaron Rodgers, GB $4 13 6. (8) Ezekiel Elliott, DAL $50 7 83. (276) Dwayne Washington, NO $0 6 73. (164) Amon-Ra St. Brown, DET $0 9'
    running_back_match = re.search(r"\) .+(?=,)", line_split[2]).group()
    running_back_names.append(running_back_match[2:])

    #parse out ADP
    ADP_match = re.search(r"\(.+(?=,)", line_split[2]).group()
    ADP.append(ADP_match[1:])
    print(ADP)
 
    if line == lines[20]:
        break

 (1) Christian McCaffrey, CAR $62 13 78
 (2) Dalvin Cook, MIN $59 7 79
 (3) Alvin Kamara, NO $59 6 80
 (4) Saquon Barkley, NYG $58 10 81
 (5) Derrick Henry, TEN $56 13 82
 (8) Ezekiel Elliott, DAL $50 7 83
 (9) Jonathan Taylor, IND $48 14 84
 (10) Austin Ekeler, LAC $46 7 85
 (11) Aaron Jones, GB $45 13 76
 (12) Nick Chubb, CLE $45 13 77
 (14) Antonio Gibson, WAS $43 9 1
 (15) Najee Harris, PIT $42 7 2
 (22) Joe Mixon, CIN $33 10 3
 (23) Clyde Edwards-Helaire, KC $32 12 4
 (30) David Montgomery, CHI $26 10 5
 (31) James Robinson, JAC $26 7 6
 (32) Miles Sanders, PHI $25 14 7
 (36) D'Andre Swift, DET $22 9 8
 (37) Chris Carson, SEA $22 9 9
 (47) Josh Jacobs, LV $15 8 10
 (48) Chase Edmonds, ARI $14 12 11
